In [20]:
import numpy as np
import pandas as pd
import sklearn

In [21]:
data=pd.read_csv('IndiaAgricultureCropProduction2.csv')

In [22]:
data

,State,District,Crop,Year,Season,Area,Area Units,Production,Production Units,Yield
0,Andaman and Nicobar Islands,NICOBARS,Arecanut,2001-02,Kharif,1254.0,Hectare,2061.0,Tonnes,1.643541
1,Andaman and Nicobar Islands,NICOBARS,Arecanut,2002-03,Whole Year,1258.0,Hectare,2083.0,Tonnes,1.655803
2,Andaman and Nicobar Islands,NICOBARS,Arecanut,2003-04,Whole Year,1261.0,Hectare,1525.0,Tonnes,1.209358
3,Andaman and Nicobar Islands,NORTH AND MIDDLE ANDAMAN,Arecanut,2001-02,Kharif,3100.0,Hectare,5239.0,Tonnes,1.690000
4,Andaman and Nicobar Islands,SOUTH ANDAMANS,Arecanut,2002-03,Whole Year,3105.0,Hectare,5267.0,Tonnes,1.696296
...,...,...,...,...,...,...,...,...,...,...
675503,West Bengal,PURBA BARDHAMAN,Wheat,2000-01,Rabi,6310.0,Hectare,15280.0,Tonnes,2.421553
675504,West Bengal,PURULIA,Wheat,1997-98,Rabi,1895.0,Hectare,2760.0,Tonnes,1.456464
675505,West Bengal,PURULIA,Wheat,1998-99,Rabi,3736.0,Hectare,5530.0,Tonnes,1.480193
675506,West Bengal,PURULIA,Wheat,1999-00,Rabi,2752.0,Hectare,6928.0,Tonnes,2.517442


In [23]:
data['Production Units'].unique()

array(['Tonnes', 'Nuts', 'Bales', nan], dtype=object)

In [24]:
data['Area Units'].unique()

array(['Hectare', nan], dtype=object)

In [25]:
data.drop('Area Units',axis=1,inplace=True)
data.drop('Year',axis=1,inplace=True)
data.drop('State',axis=1,inplace=True)
data.drop('District',axis=1,inplace=True)
data.drop('Season',axis=1,inplace=True)




In [26]:
data.isnull().any()

Crop                True
Area                True
Production          True
Production Units    True
Yield               True
dtype: bool

In [27]:
data['Crop'].isnull().sum()

50

In [28]:
data.dropna(inplace=True)

lost ~ 1.6% data due to dropping na values

In [29]:
data.isnull().any()

Crop                False
Area                False
Production          False
Production Units    False
Yield               False
dtype: bool

In [30]:
data.columns

Index(['Crop', 'Area', 'Production', 'Production Units', 'Yield'], dtype='object')

In [31]:
data

,Crop,Area,Production,Production Units,Yield
0,Arecanut,1254.0,2061.0,Tonnes,1.643541
1,Arecanut,1258.0,2083.0,Tonnes,1.655803
2,Arecanut,1261.0,1525.0,Tonnes,1.209358
3,Arecanut,3100.0,5239.0,Tonnes,1.690000
4,Arecanut,3105.0,5267.0,Tonnes,1.696296
...,...,...,...,...,...
675503,Wheat,6310.0,15280.0,Tonnes,2.421553
675504,Wheat,1895.0,2760.0,Tonnes,1.456464
675505,Wheat,3736.0,5530.0,Tonnes,1.480193
675506,Wheat,2752.0,6928.0,Tonnes,2.517442


In [32]:
data.dtypes

Crop                 object
Area                float64
Production          float64
Production Units     object
Yield               float64
dtype: object

In [33]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
# State=LE.fit_transform(data['State'])
# data['State']=State
# District=LE.fit_transform(data['District'])
# data['District']=District
Crop=LE.fit_transform(data['Crop'])
data['Crop']=Crop
# Season=LE.fit_transform(data['Season'])
# data['Season']=Season
# Area_Units=LE.fit_transform(data['Area Units'])
# data['Area Units']=Area_Units
Production_Units=LE.fit_transform(data['Production Units'])
data['Production Units']=Production_Units

In [34]:
from sklearn.model_selection import train_test_split
x=data.drop('Yield',axis=1)
y=round(data['Yield'],2)
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=1,train_size=.75)


In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor

In [36]:
randforestparams={'n_estimators':[10,20,30,40,50]}
model_details={   
                # 'linear'   :{'model':LinearRegression(),'params':{}},
                'rf'       :{'model':RandomForestClassifier(),'params':randforestparams},
              'logistics':{'model':LogisticRegression(),'params':{}},
                'dtc'      :{'model':DecisionTreeClassifier(),'params':{}},
               # 'dtc'      :{'model':DecisionTreeRegressor(),'params':{}},

             }

In [37]:
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import cross_val_score
def get_best(x,y):
    result=[]
    for model_name in model_details:
        model_dict=model_details.get(model_name)
        model=model_dict.get('model')
        params=model_dict.get('params')
        cv=GridSearchCV(model,params,return_train_score=True,cv=5)
        cv.fit(x,y)
        bp=cv.best_params_
        bs=cv.best_score_
        test_res={'model':model_name,'best_params':bp,'best_score':bs}
        result.append(test_res)
    res_df=pd.DataFrame(result)
    best_model=res_df[res_df['best_score']==res_df['best_score'].max()]
    return res_df,best_model

In [38]:
res_df,best_model=get_best(x,y)

C:\Users\ZAYYU\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ZAYYU\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ZAYYU\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "C:\Users\ZAYYU\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 734, in _validate_y_class_weight
    check_classification_targ

ValueError: Unknown label type: 'continuous'

In [ ]:
model=RandomForestClassifier()
model.fit(xtrain,ytrain)
model.score(xtest,ytest)